# Variable Selection: Forward Best Subset Selection

My own implementation of forward selection of variables. This was the first larger script i worte in R, but it get's the job done. 

In [1]:
library(tidyverse)
library(caret)
library(MLmetrics)
library(magrittr)
source("helpers.r")


heart <- get_training_df_clean()

set.seed(25)
number_of_folds <- 10
folds <- createFolds(heart$target, k = number_of_folds)

number_of_variables_to_select <- 13
variables_left <- names(heart)
variables_left <- variables_left[variables_left != "target"]
number_of_variables_initial <- length(variables_left)


result <- data.frame(matrix(ncol = 2, nrow = 0))
colnames(result) <- c("model", "auc")
result <- as_tibble(result)

formula_previous <- paste("target ~")
for(i in c(1:number_of_variables_to_select)) {
  auc_avg <- vector("numeric", length = length(number_of_variables_to_select))
  auc_avg <- 0
  
  
  for(var in variables_left){
    aucs <- vector("numeric", length = number_of_folds)
    
    if(length(variables_left) == number_of_variables_initial){
      formula_current <-  paste(formula_previous, var)
    }
    else
      formula_current <-  paste(formula_previous, "+", var)
    
    
    for(fold_index in c(1:number_of_folds)){
      training <- heart[-folds[[fold_index]],]
      test <- heart[folds[[fold_index]],]
      
      y_true <- ifelse(test$target == "no_disease", 0, 1)
      
      fit <- glm(formula_current, data=training, family =binomial(link = "logit"))
      y_probabilities <- unname(predict(fit, test,  type="response"))
      aucs[fold_index] <-  AUC(y_true = y_true, y_pred = y_probabilities)
      
    }
    
    auc_avg_current <- mean(aucs)
    
    if(auc_avg < auc_avg_current){
      aucs_selected <- aucs
      formula_selected <- formula_current
      variable_selected <- var
      
      auc_avg <- auc_avg_current
    }
    
  }
  
  result %<>%
    add_row(model = formula_selected, auc = aucs_selected)
  
  formula_previous <- formula_selected
  variables_left <- variables_left[variables_left != variable_selected]
}

(summary <- result %>%
  group_by(model) %>%
  summarize(AVG = mean(auc),
            Median = median(auc)))

Warning message:
"package 'tidyverse' was built under R version 3.6.1"Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
v ggplot2 3.1.1       v purrr   0.3.2  
v tibble  2.1.1       v dplyr   0.8.0.1
v tidyr   0.8.3       v stringr 1.4.0  
v readr   1.3.1       v forcats 0.4.0  
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
Warning message:
"package 'caret' was built under R version 3.6.1"Loading required package: lattice

Attaching package: 'caret'

The following object is masked from 'package:purrr':

    lift

Warning message:
"package 'MLmetrics' was built under R version 3.6.1"
Attaching package: 'MLmetrics'

The following objects are masked from 'package:caret':

    MAE, RMSE

The following object is 

model,AVG,Median
target ~ thal,0.7652697,0.7821678
target ~ thal + ca,0.8547952,0.8706294
target ~ thal + ca + cp,0.8908791,0.8888112
target ~ thal + ca + cp + slope,0.9128821,0.9242008
target ~ thal + ca + cp + slope + oldpeak,0.9166983,0.9429321
target ~ thal + ca + cp + slope + oldpeak + sex,0.9197053,0.9359391
target ~ thal + ca + cp + slope + oldpeak + sex + trestbps,0.9227572,0.9400100
target ~ thal + ca + cp + slope + oldpeak + sex + trestbps + restecg,0.9204496,0.9358142
target ~ thal + ca + cp + slope + oldpeak + sex + trestbps + restecg + thalach,0.9180020,0.9428072
target ~ thal + ca + cp + slope + oldpeak + sex + trestbps + restecg + thalach + chol,0.9166034,0.9393107


In [2]:
(best_model <-  as.character(summary[summary$AVG == max(summary$AVG),1]))

[1] "target ~ thal + ca + cp + slope + oldpeak + sex + trestbps"